# Actor Critic Agent(s) for Resource Allocation
- Tests with Gym first
- https://pytorch.org/docs/stable/tensorboard.html

In [290]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import gym

from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
import gym.spaces as spaces
from torch.distributions import Categorical
import torch.optim as optim
import stable_baselines3

## Actor-Critic
- https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html

In [ ]:
NUM_HIDDEN_UNITS = 32

In [ ]:
class CategoricalWrapper(torch.distributions.Categorical):
    def log_probs(self, actions):
        return super().log_prob(actions.squeeze(-1)).view(actions.size(0), -1).sum(-1).unsqueeze(-1)
    
    def sample(self):
        return super().sample().unsqueeze(-1)


In [ ]:
class ActorCritic(nn.Module):
    def __init__(self, obs_shape, action_space, discrete=True):
        super(ActorCritic, self).__init__()
        
        self.obs_shape = obs_shape
        self.action_space = action_space
        
        self._is_discrete = discrete
        self._num_inputs = obs_shape[0]
        self._n_hidden = NUM_HIDDEN_UNITS
        if discrete:
            self._num_outputs = action_space.n
        else:
            self._num_outputs = action_space.shape[0]
        
        self.actor = nn.Sequential(
            nn.Linear(self._num_inputs, self._n_hidden),
            nn.Tanh(),
            nn.Linear(self._n_hidden, self._n_hidden),
            nn.Tanh(),
            nn.Linear(self._n_hidden, self._num_outputs)
        )
        
        self.critic = nn.Sequential(
            nn.Linear(self._num_inputs, self._n_hidden),
            nn.Tanh(),
            nn.Linear(self._n_hidden, self._n_hidden),
            nn.Tanh(),
            nn.Linear(self._n_hidden, 1)
        )
        
        self.train()        
    
    def forward(self, x):
        # forward through networks
        actions = self.actor(x)
        critique = self.critic(x)
        
        return actions, critique
    
    def act(self, x):
        actions, value = self(x)
        
        # sample
        dist = CategoricalWrapper(logits=actions)
        action = dist.sample()
        
        # generate log probabilities
        log_prob = dist.log_probs(action)
        return value, action, log_prob
        
    def evaluate_action(self, x, action):
        actions, value = self(x)
        
        dist = CategoricalWrapper(logits=actions)
        
        # generate log probabilities
        log_prob = dist.log_probs(action)
        return value, log_prob
    
    def get_value(self, x):
        _, value = self(x)
        return value


In [ ]:
# Adapted from Ikostrikov's A2C Rollout Storage
class RolloutStorage(object):
    def __init__(self, num_steps, obs_shape, action_space, num_processes=1):
        self.obs = torch.zeros(num_steps + 1, num_processes, *obs_shape)
        
        self.rewards = torch.zeros(num_steps, num_processes, 1)
        self.value_preds = torch.zeros(num_steps + 1, num_processes, 1)
        
        self.returns = torch.zeros(num_steps + 1, num_processes, 1)
        self.action_log_probs = torch.zeros(num_steps, num_processes, 1)
        
        if action_space.__class__.__name__ == 'Discrete':
            action_shape = 1
        else:
            action_shape = action_space.shape[0]
        
        self.actions = torch.zeros(num_steps, num_processes, action_shape)
        
        if action_space.__class__.__name__ == 'Discrete':
            self.actions = self.actions.long()
        
        self.masks = torch.ones(num_steps + 1, num_processes, 1)

        # Masks that indicate whether it's a true terminal state
        # or time limit end state
        # self.bad_masks = torch.ones(num_steps + 1, num_processes, 1)

        self.num_steps = num_steps
        self.step = 0

    def to(self, device):
        self.obs = self.obs.to(device)
        
        self.rewards = self.rewards.to(device)
        self.value_preds = self.value_preds.to(device)
        self.returns = self.returns.to(device)
        
        self.action_log_probs = self.action_log_probs.to(device)
        self.actions = self.actions.to(device)
        self.masks = self.masks.to(device)

    def insert(self, obs, actions, action_log_probs, value_preds, rewards, masks):
        self.obs[self.step + 1].copy_(obs)
        
        self.actions[self.step].copy_(actions)
        self.action_log_probs[self.step].copy_(action_log_probs)
        self.value_preds[self.step].copy_(value_preds)
        
        self.rewards[self.step].copy_(rewards)
        self.masks[self.step + 1].copy_(masks)

        self.step = (self.step + 1) % self.num_steps

    def after_update(self):
        self.obs[0].copy_(self.obs[-1])
        self.masks[0].copy_(self.masks[-1])

    def compute_returns(self, next_value, gamma):
        self.returns[-1] = next_value
        
        for step in reversed(range(self.rewards.size(0))):
            self.returns[step] = (self.returns[step + 1] * \
                gamma * self.masks[step + 1] + self.rewards[step])


In [ ]:
class Optimizer:
    def __init__(self, model, lr=7e-4, eps=1e-5, alpha=0.99):
        self.optimizer = optim.RMSprop(model.parameters(), lr, eps=eps, alpha=alpha)
        self.model = model
    
    # adapted from Ikostrikov's A2C
    def update(self, rollouts):
        obs_shape = rollouts.obs.size()[2:]
        action_shape = rollouts.actions.size()[-1]
        num_steps, num_processes, _ = rollouts.rewards.size()
        
        values, action_log_probs = self.model.evaluate_action(
            rollouts.obs[:-1].view(-1, *obs_shape),
            rollouts.actions.view(-1, action_shape)
        )

        values = values.view(num_steps, num_processes, 1)
        action_log_probs = action_log_probs.view(num_steps, num_processes, 1)

        advantages = rollouts.returns[:-1] - values
        value_loss = advantages.pow(2).mean()

        action_loss = -(advantages.detach() * action_log_probs).mean()
        self.optimizer.zero_grad()
        (value_loss + action_loss).backward()
        
        nn.utils.clip_grad_norm_(self.model.parameters(), 0.5)

        self.optimizer.step()

        return value_loss.item(), action_loss.item()


In [ ]:
a = ActorCritic((3, 1), spaces.Discrete(5))
print(a)
o = Optimizer(a, 0.01)

ActorCritic(
  (actor): Sequential(
    (0): Linear(in_features=3, out_features=32, bias=True)
    (1): Tanh()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): Tanh()
    (4): Linear(in_features=32, out_features=5, bias=True)
  )
  (critic): Sequential(
    (0): Linear(in_features=3, out_features=32, bias=True)
    (1): Tanh()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): Tanh()
    (4): Linear(in_features=32, out_features=1, bias=True)
  )
)


## Gym Environment Test

In [291]:
env = gym.make('CartPole-v1')
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
print(device)

agent = ActorCritic(env.observation_space.shape, env.action_space)
agent.to(device)

opt = Optimizer(agent)

rollouts = RolloutStorage(5, env.observation_space.shape, env.action_space)
rollouts.to(device)

cuda


In [ ]:
t = 0
history = []
M = 1

first_time = True

obs = env.reset()
rollouts.obs[0].copy_(torch.from_numpy(obs))
rollouts.to(device)

cur_step = 0

for i in range(5000):
    if not first_time:
        obs = env.reset()
    else:
        first_time = False
    
    done = False
    total_reward = 0
    n = 1

    while not done:
        value, action, log_prob = agent.act(rollouts.obs[cur_step])
        next_obs, reward, done, _ = env.step(action.item())
        next_obs = torch.from_numpy(next_obs)
        
        total_reward += reward
        
        masks = torch.FloatTensor([[0.0] if done else [1.0]])
        
        rollouts.insert(next_obs.clone(), action, log_prob, value, reward, masks.clone())
        
        if cur_step == 5:
            cur_step = 0
            with torch.no_grad():
                next_value = agent.get_value(rollouts.obs[-1]).detach()
                
            rollouts.compute_returns(next_value, 0.99)
            value_loss, action_loss = opt.update(rollouts)
            rollouts.after_update()
        
        cur_step += 1
        
        obs = next_obs


    history.append(total_reward)
    # self.episodes.append(i)
    mean_time = np.mean(history[-10:])
    # self.ep_decays.append(self.epsilon)
    # self.epsilon = max(0.001, self.epsilon*self.ep_decay)
   #  self.total_reward_ep.append(total_reward)
    if i % 100 == 0:
        print('At episode', i, ': mean reward of', mean_time)
    if mean_time > 480.:
        print('Optimal Agent Achieved')
        break

print('Finished Training')

At episode 0 : mean reward of 10.0
At episode 100 : mean reward of 17.3
At episode 200 : mean reward of 11.9
At episode 300 : mean reward of 11.6
At episode 400 : mean reward of 15.7
At episode 500 : mean reward of 16.0
At episode 600 : mean reward of 9.9
At episode 700 : mean reward of 13.3
At episode 800 : mean reward of 11.7
At episode 900 : mean reward of 10.0
At episode 1000 : mean reward of 10.0
At episode 1100 : mean reward of 11.4
At episode 1200 : mean reward of 9.7
At episode 1300 : mean reward of 11.6
At episode 1400 : mean reward of 9.7
At episode 1500 : mean reward of 13.3
At episode 1600 : mean reward of 9.5
At episode 1700 : mean reward of 14.7
At episode 1800 : mean reward of 9.7
At episode 1900 : mean reward of 9.9
At episode 2000 : mean reward of 12.2
At episode 2100 : mean reward of 9.4
At episode 2200 : mean reward of 9.5
At episode 2300 : mean reward of 16.2
At episode 2400 : mean reward of 11.5
At episode 2500 : mean reward of 9.8
At episode 2600 : mean reward of 

KeyboardInterrupt: 

In [ ]:
from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_vec_env

env = make_vec_env("CartPole-v1", n_envs=4)

model = A2C("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=25000)
model.save("a2c_cartpole")

Using cuda device
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 39.5     |
|    ep_rew_mean        | 39.5     |
| time/                 |          |
|    fps                | 5711     |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 2000     |
| train/                |          |
|    entropy_loss       | -0.61    |
|    explained_variance | 0.155    |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 1.58     |
|    value_loss         | 8.04     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 58.3     |
|    ep_rew_mean        | 58.3     |
| time/                 |          |
|    fps                | 5972     |
|    iterations         | 200      |
|    time_elapsed       | 0        |
|    total_timesteps    | 4000     |
| train/            

## Environment

In [296]:
class SustainableEnvironment(gym.Env):
    def __init__(self, max_land_resources = 100, max_food_storage = 200, disaster_probability = 0.05):
        self.action_space = spaces.Discrete(11)
        self.max_land_resources = max_land_resources
        self.max_food_storage = max_food_storage
        self.disaster_probability = disaster_probability
        self.high = np.array(
            [
                2*self.max_land_resources,
                self.max_land_resources,
                self.max_food_storage
            ],
            dtype=np.int32,
        )
        self.low = np.array(
            [
                0,
                0,
                0
            ],
            dtype=np.int32
        )
        self.observation_space = spaces.Box(self.low, self.high, dtype=np.int32)
        self.food_storage = 0
        self.resource_recovery = {
            0:100,
            1:95,
            2:90,
            3:85,
            4:80,
            5:75,
            6:70,
            7:65,
            8:60,
            9:55,
            10:50
        }
    
    def reset(self):
        # self.state = [random.randint(11, 2*self.max_land_resources), random.randint(11, self.max_land_resources), random.randint(0, self.max_food_storage)]
        self.state = [50, 100, 0]
        return np.array(self.state, dtype=np.int32)
    
    def step(self, action):
        population, land_resources, food_storage = self.state
        done = False
        reward = 0
        
        percent_land_use = action/10
        land_utilized = food_produced =  int(land_resources * percent_land_use)
        # print(f'land_utilized: {land_utilized}')
        land_resources = min(self.max_land_resources, (land_resources - land_utilized) + (land_resources - land_utilized) * self.resource_recovery[action]/100)
        # print(f'land_resources: {land_resources}')
        # Dividng the land resources thing. More land, more utilization, less recovery. More land, less utilization, more recovery.
        # Less land, more utilization, 
        people_died = 0
        if(population > food_produced):
            if(population > food_storage + food_produced):
                # Some people will die. Negative reward should be given.
                people_died = population - food_storage - food_produced
                # print(f'people_died_hunger: {people_died}')
                population = food_storage + food_produced
                # print(f'new population for now: {population}')
                food_storage = 0
                reward -= people_died
            else:
                # No one should die of hunger. Some food will be taken from fresh produce
                # and some will be taken from storage. Population remains the same for now.
                food_storage -= (population - food_produced)
                # print(f'food_storage: {food_storage}')
        else:
            # No one should die. All the demand will be satisfied by fresh produce.
            # Leftover food will be added to storage. Population remains the same for now.
            food_storage += (food_produced - population)
            # print(f'food_storage: {food_storage}')
        food_storage_overflow = max(0, food_storage - self.max_food_storage)
        # print(f'food_storage_overflow: {food_storage_overflow}')
        reward -= food_storage_overflow
        # Negative reward should be given if overflow occurs, because such behavior is discouraged.
        food_storage = min(self.max_food_storage, food_storage)
        # print(f'food_storage: {food_storage}')
        # How much does each of the state variables decrease? Two options: either naturally, which is 5%-10%, or due to a disaster, 
        # which will be 50% to 70% for food storage, 30% to 50% for population and 10% to 20% for land.
        disaster = np.random.choice(2, p = [self.disaster_probability, 1 - self.disaster_probability]) == 0
        # print(disaster)
        if(disaster):
            # Disaster occured this year
            # Decreasing all the state values first
            percent_food_spoiled = random.uniform(0.5, 0.7)
            # print(f'percent_food_spoiled: {percent_food_spoiled}')
            food_decrease = int(round(food_storage * percent_food_spoiled))
            # print(f'food_decrease: {food_decrease}')
            population_decrease_factor = random.uniform(0.3, 0.5)
            # print(f'population_decrease_factor: {population_decrease_factor}')
            population_decrease = int(round(population * population_decrease_factor))
            # print(f'population_decrease: {population_decrease}')
            land_decrease_factor = random.uniform(0.1, 0.2)
            # print(f'land_decrease_factor: {land_decrease_factor}')
            land_decrease = int(round(land_resources * land_decrease_factor))
            # print(f'land_decrease: {land_decrease}')
            
            # Now increasing values for disaster year. Assuming population will not increase by a lot in a disaster.
            population_increase_factor = random.uniform(0.01, 0.20)
            # print(f'population_increase_factor: {population_increase_factor}')
            population_increase = int(round(population * population_increase_factor))
            # print(f'population_increase: {population_increase}')
            
        else:
            # Disaster did not occur this year. Land does not deteriorate over time, it only gets better for farming.
            percent_food_spoiled = random.uniform(0.05, 0.10)
            # print(f'percent_food_spoiled: {percent_food_spoiled}')
            food_decrease = int(round(food_storage * percent_food_spoiled))
            # print(f'food_decrease: {food_decrease}')
            population_decrease_factor = random.uniform(0.05, 0.10)
            # print(f'population_decrease_factor: {population_decrease_factor}')
            population_decrease = int(round(population * population_decrease_factor))
            # print(f'population_decrease: {population_decrease}')
            land_decrease_factor = 0
            # print(f'land_decrease_factor: {land_decrease_factor}')
            land_decrease = int(land_resources * land_decrease_factor)
            
            # Increasing state values according to if disaster did not occur.
            population_increase_factor = random.uniform(0.05, 0.20)
            # print(f'population_increase_factor: {population_increase_factor}')
            population_increase = int(round(population * population_increase_factor))
            # print(f'population_increase: {population_increase}')
            
        population = max(0, population - population_decrease + population_increase)
        # print(f'population: {population}')
        food_storage = max(0, food_storage - food_decrease)
        # print(f'food_storage: {food_storage}')
        land_resources = max(0, land_resources - land_decrease)
        # print(f'land_resources: {land_resources}')
        
        if(population < 10 or land_resources < 10):
            done = True
        if(people_died == 0 and food_storage_overflow == 0):
            reward += population + land_resources
        
        
        self.state = np.array([population, land_resources, food_storage], dtype=np.int32)
        return self.state, reward, done, {}
    
    def render(self):
        pass

In [297]:
env = SustainableEnvironment()
obs = env.reset()
done = False
# while not done:
#     print(f'Observation: {obs}')
#     action = np.random.choice(11)
#     print(f'Action: {action}')
#     new_obs, reward, done = env.step(action)
#     print(f'New Obs: {new_obs}')
#     obs = new_obs
print(obs)
env.step(1)
    

[ 50 100   0]


(array([ 11, 100,   0], dtype=int32), -40, False, {})

In [298]:
from stable_baselines3.common.env_checker import check_env

In [299]:
check_env(env)

In [302]:
from gym.envs.registration import register
# Example for the CartPole environment
register(
    # unique identifier for the env `name-version`
    id="SustainableEnvironment-v1",
    # path to the class for creating the env
    # Note: entry_point also accept a class as input (and not only a string)
    entry_point=SustainableEnvironment,
    # Max number of steps per episode, using a `TimeLimitWrapper`
    max_episode_steps=250,
)

In [303]:
from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_vec_env

env = make_vec_env("SustainableEnvironment-v1", n_envs=4)

model = A2C("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=25000)
model.save("a2c_sustainable")

Using cuda device
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 8.38     |
|    ep_rew_mean        | 410      |
| time/                 |          |
|    fps                | 5017     |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 2000     |
| train/                |          |
|    entropy_loss       | -1.85    |
|    explained_variance | 0.0206   |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 72.2     |
|    value_loss         | 2.01e+04 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 15.1     |
|    ep_rew_mean        | 927      |
| time/                 |          |
|    fps                | 5135     |
|    iterations         | 200      |
|    time_elapsed       | 0        |
|    total_timesteps    | 4000     |
| train/            